# Trading Dashboard

---

### Import Libraries and Dependencies

In [25]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
from pathlib import Path
import plotly.express as px
import panel as pn

pn.extension()

import hvplot
import hvplot.pandas

pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.width", 1000)

In [26]:
# For reading stock data from yahoo
from pandas_datareader.data import DataReader

# For time stamps
from datetime import datetime

In [27]:
# Set up End and Start times for data grab
years = 10
end = datetime.now()
start = datetime(end.year - years, end.month, end.day)

# Identify stock data to grab by ticker
ticker = 'AAPL'

In [28]:
# Read in stock data
aapl_df = DataReader(ticker, 'yahoo', start, end)

In [29]:
# Save stock data to a csv file
aapl_df.to_csv(r'AAPL1.csv')

### Read CSV into Pandas DataFrame

In [30]:
# Set the file path
filepath = Path("AAPL1.csv")

# Read the CSV located at the file path into a Pandas DataFrame
aapl_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)

# Print the DataFrame
aapl_df.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2011-01-10,12.258214,12.041786,12.101071,12.230357,448560000.0,10.550527
1,2011-01-11,12.320000,12.123929,12.317143,12.201429,444108000.0,10.525572
2,2011-01-12,12.301071,12.214286,12.258929,12.300714,302590400.0,10.611217
3,2011-01-13,12.380000,12.280357,12.327143,12.345714,296780400.0,10.650034
4,2011-01-14,12.445714,12.301429,12.353214,12.445714,308840000.0,10.736302


### Generate a Dual Moving Average Crossover Trading Signal

In [31]:
# Grab just the `date` and `close` from the dataset
signals_df = aapl_df.loc[:, ['Date', 'Close']].copy()

# Set the `date` column as the index
signals_df = signals_df.set_index("Date", drop=True)

# Set the short window and long windows
short_window = 50
long_window = 100

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df['SMA50'] = signals_df['Close'].rolling(window=short_window).mean()
signals_df['SMA100'] = signals_df['Close'].rolling(window=long_window).mean()
signals_df['Signal'] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA50 is under the SMA100, and
# where 1 is when the SMA50 is higher (or crosses over) the SMA100
signals_df['Signal'][short_window:] = np.where(
    signals_df['SMA50'][short_window:] > signals_df['SMA100'][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df['Entry/Exit'] = signals_df['Signal'].diff()

# Print the DataFrame
signals_df.tail(10)

,Close,SMA50,SMA100,Signal,Entry/Exit
Date,,,,,
2020-12-24,131.970001,119.7370,118.223525,1.0,0.0
2020-12-28,136.690002,120.0566,118.489800,1.0,0.0
2020-12-29,134.869995,120.3736,118.699475,1.0,0.0
2020-12-30,133.720001,120.7284,118.925550,1.0,0.0
2020-12-31,132.690002,121.0320,119.125175,1.0,0.0
2021-01-04,129.410004,121.2828,119.325525,1.0,0.0
2021-01-05,131.009995,121.5880,119.505525,1.0,0.0
2021-01-06,126.599998,121.8192,119.621425,1.0,0.0
2021-01-07,130.919998,122.1366,119.781550,1.0,0.0


### Plot Entry and Exit Points of Dual Moving Average Crossover Trading Strategy

In [32]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

### Backtest the Trading Strategy

In [33]:
# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 500

# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
signals_df['Position'] = share_size * signals_df['Signal']

# Find the points in time where a 500 share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df['Portfolio Holdings'] = signals_df['Close'] * signals_df['Entry/Exit Position'].cumsum()

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df['Portfolio Cash'] = initial_capital - (signals_df['Close'] * signals_df['Entry/Exit Position']).cumsum()

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()

# Calculate the cumulative returns
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1

# Print the DataFrame
signals_df.tail(10)

,Close,SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,
2020-12-24,131.970001,119.7370,118.223525,1.0,0.0,500.0,0.0,65985.000610,78563.572884,144548.573494,0.003506,0.445486
2020-12-28,136.690002,120.0566,118.489800,1.0,0.0,500.0,0.0,68345.001221,78563.572884,146908.574104,0.016327,0.469086
2020-12-29,134.869995,120.3736,118.699475,1.0,0.0,500.0,0.0,67434.997559,78563.572884,145998.570442,-0.006194,0.459986
2020-12-30,133.720001,120.7284,118.925550,1.0,0.0,500.0,0.0,66860.000610,78563.572884,145423.573494,-0.003938,0.454236
2020-12-31,132.690002,121.0320,119.125175,1.0,0.0,500.0,0.0,66345.001221,78563.572884,144908.574104,-0.003541,0.449086
2021-01-04,129.410004,121.2828,119.325525,1.0,0.0,500.0,0.0,64705.001831,78563.572884,143268.574715,-0.011317,0.432686
2021-01-05,131.009995,121.5880,119.505525,1.0,0.0,500.0,0.0,65504.997253,78563.572884,144068.570137,0.005584,0.440686
2021-01-06,126.599998,121.8192,119.621425,1.0,0.0,500.0,0.0,63299.999237,78563.572884,141863.572121,-0.015305,0.418636
2021-01-07,130.919998,122.1366,119.781550,1.0,0.0,500.0,0.0,65459.999084,78563.572884,144023.571968,0.015226,0.440236


### Plot Entry/Exit Points of Trading Strategy vs. Backtest Results

In [34]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

### Prepare Portfolio Evaluation Metrics DataFrame

In [35]:
# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` (just like PyFolio)
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


### Calculate and Assign Portfolio Evaluation Metrics

In [36]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]

# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)

# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2
portfolio_evaluation_df

# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df.head()

,Backtest
Annual Return,0.0386075
Cumulative Returns,0.445886
Annual Volatility,0.0576163
Sharpe Ratio,0.67008
Sortino Ratio,0.976964


### Prepare Trade Evaluation Metrics DataFrame

In [37]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


### Generating Trade Evaluation Metrics

In [38]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = abs(row['Portfolio Holdings'])
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Close'] * row['Entry/Exit Position'])
        exit_share_price = row['Close']
        profit_loss =  entry_portfolio_holding - exit_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'AAPL',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,AAPL,2011-07-26,2012-07-06,500.0,14.407500,21.638571,7203.750134,10819.285393,-3615.535259
1,AAPL,2012-08-07,2012-11-21,500.0,22.175358,20.060715,11087.678909,10030.357361,1057.321548
2,AAPL,2013-07-10,2013-07-16,500.0,15.026071,15.364286,7513.035297,7682.143211,-169.107914
3,AAPL,2013-08-15,2014-03-10,500.0,17.782499,18.961430,8891.249657,9480.714798,-589.465141
4,AAPL,2014-05-06,2015-08-03,500.0,21.228930,29.610001,10614.464760,14805.000305,-4190.535545
5,AAPL,2015-12-09,2016-01-13,500.0,28.905001,24.347500,14452.500343,12173.749924,2278.750420
6,AAPL,2016-04-25,2016-06-14,500.0,26.270000,24.365000,13135.000229,12182.499886,952.500343
7,AAPL,2016-08-22,2018-03-26,500.0,27.127501,43.192501,13563.750267,21596.250534,-8032.500267
8,AAPL,2018-04-19,2018-11-30,500.0,43.200001,44.645000,21600.000381,22322.500229,-722.499847
9,AAPL,2019-03-28,2019-07-22,500.0,47.180000,51.805000,23590.000153,25902.500153,-2312.500000


---

### Create Hvplot Line Chart of Closing, SMA50, and SMA100

In [39]:
price_df = signals_df[['Close', 'SMA50', 'SMA100']]
price_chart = price_df.hvplot.line()
price_chart.opts(xaxis=None)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

### Create Hvplot Table of Portfolio Metrics

In [40]:
portfolio_evaluation_df.reset_index(inplace=True)
portfolio_evaluation_table = portfolio_evaluation_df.hvplot.table()
portfolio_evaluation_table

:Table   [index,Backtest]

### Create Hvplot Table of Trade Metrics

In [41]:
trade_evaluation_table = trade_evaluation_df.hvplot.table()
trade_evaluation_table

:Table   [Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss]

### Build a Dashboard with Panel

In [42]:
# Create rows
price_chart_row = pn.Row(price_chart)
portfolio_evaluation_row = pn.Row(portfolio_evaluation_table)
trade_evaluation_row = pn.Row(trade_evaluation_table)

# Create columns
portfolio_column = pn.Column('# Portfolio Evaluation Metrics', price_chart_row, portfolio_evaluation_row)
trade_column = pn.Column('# Trade Evaluation Metrics', trade_evaluation_row)

# Create tabs
trading_dashboard = pn.Tabs(
    ("Portfolio Metrics", portfolio_column),
    ("Trade Metrics", trade_column)
)

### Serve Panel Dashboard on Local Host

In [43]:
trading_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(NdOverlay)
        [2] Row
            [0] HoloViews(Table)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(Table)